In [1]:
import os
import re
from copy import deepcopy
from pathlib import Path

import cheetah.bmad

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [3]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [5]:
lines = cheetah.bmad.read_clean_lines(resolved_lattice_file_path)
lines[:20]

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0',
 'sethxrss = 0',
 'setsxrss = 0',
 'setcbxfel = 0',
 'setpepx = 0',
 'intgsx = 30.0']

In [8]:
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    lines, delimitter="&", remove_delimitter=True
)
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    merged_lines, delimitter=",", remove_delimitter=False
)
merged_lines = cheetah.bmad.merge_delimitter_continued_lines(
    merged_lines, delimitter="{", remove_delimitter=False
)
len(lines), len(merged_lines)

(14409, 12215)

In [10]:
property_assignment_pattern = r"[a-z0-9_]+\[[a-z0-9_%]+\]\s*=.*"
variable_assignment_pattern = r"[a-z0-9_]+\s*=.*"
element_definition_pattern = r"[a-z0-9_]+\:.*"
line_definition_pattern = r"[a-z0-9_]+\s*\:\s*line\s*=\s*\(.*\)"
use_line_pattern = r"use\s*\,\s*[a-z0-9_]+"

num_successful = 0
num_property_assignment = 0
num_variable_assignment = 0
num_element_definition = 0
num_line_definition = 0
num_use_line = 0
for line in merged_lines:
    if re.fullmatch(property_assignment_pattern, line):
        num_successful += 1
        num_property_assignment += 1
    elif re.fullmatch(variable_assignment_pattern, line):
        num_successful += 1
        num_variable_assignment += 1
    elif re.fullmatch(element_definition_pattern, line):
        num_successful += 1
        num_element_definition += 1
    elif re.fullmatch(line_definition_pattern, line):
        num_successful += 1
        num_line_definition += 1
    elif re.fullmatch(use_line_pattern, line):
        num_successful += 1
        num_use_line += 1
    else:
        print(line)
        break

print("")
print("######################################")
print(f"num_successful: {num_successful} / {len(merged_lines)}")
print("--------------------------------------")
print(f"{num_property_assignment = }")
print(f"{num_variable_assignment = }")
print(f"{num_element_definition = }")
print(f"{num_line_definition = }")
print(f"{num_use_line = }")
print("######################################")


######################################
num_successful: 12215 / 12215
--------------------------------------
num_property_assignment = 4256
num_variable_assignment = 1684
num_element_definition = 5657
num_line_definition = 617
num_use_line = 1
######################################
